In [1]:
import re

# given original or target expression, return splitted
def split_exprs(input, output):
    input = input.strip()
    input, var = input.split(")/d")

    # exprs.append(expr[2:].replace(var, 'x'))
    
    input = input[2:]
    # Step 1: Replace `exp`, `cos`, and `sin` with placeholders
    input = input.replace('exp', 'E')
    input = input.replace('cos', 'C')
    input = input.replace('sin', 'S')
    output = output.replace('exp', 'E')
    output = output.replace('cos', 'C')
    output = output.replace('sin', 'S')

    # Step 2: Substitute `e`, `s`, `c` with 'x' (only where they’re standalone)
    input = input.replace(var, 'x')
    output = output.replace(var, 'x')

    # Step 3: Restore the placeholders back to their original words
    input = input.replace('E', 'exp')
    input = input.replace('C', 'cos')
    input = input.replace('S', 'sin')
    output = output.replace('E', 'exp')
    output = output.replace('C', 'cos')
    output = output.replace('S', 'sin')

    return input, output

# read training data line by line
def read_data(file):
    inputs = []
    outputs = []
    with open(file, 'r') as f:
        for i, line in enumerate(f):
            line = line.strip()
            
            input, output = line.split('=')
            input = input.strip()

            # split original and target expression
            input, output = split_exprs(input, output)
            
            inputs.append(input)
            outputs.append(output)

    return inputs, outputs

print(read_data("train.txt")[1][:5])

['72exp^(9x)', '340exp^(17x)+18exp^(x)', '144exp^(12x)+180exp^(20x)', '26x+36x+100x^4+114x^5', '456exp^(19x)*x+24exp^(19x)']


In [2]:

# break up the sequence to list of tokens
def add_space(expr):
    regex = '(exp\^\(|sin\(|cos\(|sin\^|cos\^|\+|\-|\(|\)|\^|\*|x|\d)'
    splitted = re.split(regex, expr)
    splitted = [x for x in splitted if x is not None and x != '']
    return splitted

X = []
y = []
inputs, outputs = read_data("train.txt")
X = [add_space(expr) for expr in inputs]
y = [add_space(expr) for expr in outputs]

print(X[:5])
print(y[:5])
    

[['8', 'exp^(', '9', 'x', ')'], ['2', '0', 'exp^(', '1', '7', 'x', ')', '+', '1', '8', 'exp^(', 'x', ')'], ['1', '2', 'exp^(', '1', '2', 'x', ')', '+', '9', 'exp^(', '2', '0', 'x', ')'], ['1', '3', 'x', '^', '2', '+', '1', '8', 'x', '^', '2', '+', '2', '0', 'x', '^', '5', '+', '1', '9', 'x', '^', '6'], ['2', '4', 'exp^(', '1', '9', 'x', ')', '*', 'x']]
[['7', '2', 'exp^(', '9', 'x', ')'], ['3', '4', '0', 'exp^(', '1', '7', 'x', ')', '+', '1', '8', 'exp^(', 'x', ')'], ['1', '4', '4', 'exp^(', '1', '2', 'x', ')', '+', '1', '8', '0', 'exp^(', '2', '0', 'x', ')'], ['2', '6', 'x', '+', '3', '6', 'x', '+', '1', '0', '0', 'x', '^', '4', '+', '1', '1', '4', 'x', '^', '5'], ['4', '5', '6', 'exp^(', '1', '9', 'x', ')', '*', 'x', '+', '2', '4', 'exp^(', '1', '9', 'x', ')']]


In [3]:
from torchtext.vocab import build_vocab_from_iterator

# Build the vocabulary for input and output
input_vocab = build_vocab_from_iterator(X, specials=["[UNK]", "[PAD]"])
input_vocab.set_default_index(input_vocab["[UNK]"])  # Set default index for OOV tokens
output_vocab = build_vocab_from_iterator(y, specials=["[UNK]", "[PAD]"])
output_vocab.set_default_index(output_vocab["[UNK]"])  # Set default index for OOV tokens

# Print vocabulary and integer encoding
print("Input Vocabulary:", input_vocab.get_stoi())      # stoi: string-to-integer mapping
print("Output Vocabulary:", output_vocab.get_stoi())    # stoi: string-to-integer mapping
print("Encoded Inputs:", [input_vocab(tokens) for tokens in X[:5]])  # Convert tokens to integer IDs
print("Encoded Outputs:", [output_vocab(tokens) for tokens in y[:5]])  # Convert tokens to integer IDs

Input Vocabulary: {'^': 4, '[UNK]': 0, 's': 33, '3': 10, '[PAD]': 1, '8': 12, 'x': 2, 'i': 32, ')': 5, '1': 3, 'o': 36, 'exp^(': 6, 'r': 24, '2': 7, 'k': 34, '+': 8, 't': 30, '4': 9, 'v': 40, '6': 11, 'p': 28, '0': 13, 'u': 25, '5': 14, 'w': 38, '7': 15, 'y': 27, '9': 16, '*': 17, '(': 18, 'sin^': 19, 'cos^': 20, 'm': 41, '-': 21, 'cos(': 22, 'sin(': 23, 'b': 26, 'z': 29, 'a': 31, 'n': 35, 'c': 37, 'e': 39}
Output Vocabulary: {'^': 5, '[UNK]': 0, 's': 33, '3': 14, '[PAD]': 1, 'p': 28, '0': 7, '8': 12, 'x': 2, 'i': 32, ')': 4, '1': 3, 'r': 24, '2': 6, '*': 8, 'o': 36, 'exp^(': 9, 't': 30, '4': 10, 'v': 40, '6': 11, 'k': 34, '+': 13, 'u': 25, '5': 15, 'w': 38, '7': 16, 'y': 27, '9': 17, '(': 18, 'cos(': 19, 'sin(': 20, 'm': 41, '-': 21, 'sin^': 22, 'cos^': 23, 'b': 26, 'z': 29, 'a': 31, 'n': 35, 'c': 37, 'e': 39}
Encoded Inputs: [[12, 6, 16, 2, 5], [7, 13, 6, 3, 15, 2, 5, 8, 3, 12, 6, 2, 5], [3, 7, 6, 3, 7, 2, 5, 8, 16, 6, 7, 13, 2, 5], [3, 10, 2, 4, 7, 8, 3, 12, 2, 4, 7, 8, 7, 13, 2, 4,

: 